In [ ]:
from bs4 import BeautifulSoup
import requests

base_url = 'https://npa.krungthai.com/api/v1'
url = base_url + '/product/searchAll'

payload = {
    "priceRangeMax": "20000000",
    "priceRangeMin": "10000",
    "shrProvince": "10",
    "typeProp": [
        "11"
    ],
    "paging": {
        "totalRows": 0,
        "rowsPerPage": 150,
        "currentPage": 1
    }
}

reqs = requests.post(url, verify=False, json=payload)
print(reqs)

In [ ]:
import pandas as pd    

df = pd.json_normalize(reqs.json()['dataResponse'])
print(df)

df.to_csv("../../dataset/npa/krungthai.csv")

In [ ]:
from datetime import datetime
import json

df = pd.read_csv("../../dataset/npa/krungthai.csv")
# for index, row in df.iterrows():
#     formatted_prices = [{
#         "date": datetime.today().strftime('%Y-%m-%d'), "price": float(row['priceNumber'])
#     }]
#     for rawprice in json.loads(row['shrPriceVos'].replace("\'", "\"")):
#         formatted_price = {"date": rawprice['year']+"-01-01", "price": float(rawprice['price'])}
#         formatted_prices.append(formatted_price)
#     df.at[index,'shrPriceVos'] = json.dumps(formatted_prices)


# df.to_csv("../../dataset/npa/krungthai.csv")

# Image can be sent using a string from "fileName" column
# Example : https://npa.krungthai.com/image/product/305645202304261550136509.jpg

In [ ]:
df = pd.read_csv("../../dataset/npa/krungthai.csv")
df.to_csv(f"../../dataset/npa/krungthai_{datetime.today().strftime('%Y-%m-%d')}.csv")

# Upsert Database

In [120]:
df = pd.json_normalize(reqs.json()['dataResponse'])
db_df = pd.read_csv("../../dataset/npa/krungthai_2023-11-01.csv")

df.set_index('collGrpId', inplace=True)
db_df.set_index('collGrpId', inplace=True)

"""SOLD properties"""
# outer_join_df = db_df.join(df, lsuffix="_sold", how='outer')
# print(outer_join_df)

# sold_properties = outer_join_df[outer_join_df["shrProvinceName"].isna()]
# print(sold_properties)

# print(db_df.columns.size)

# sold_properties = sold_properties.drop(db_df, axis=1)
# print(sold_properties)

"""NEW properties"""
# outer_join_df = db_df.join(df, rsuffix="_new", how='outer')
# print(outer_join_df)

# sold_properties = outer_join_df[outer_join_df["shrProvinceName"].isna()]
# print(sold_properties)

# print(db_df.columns.size)

# sold_properties = sold_properties.drop(db_df, axis=1)
# print(sold_properties)

"""DB UPDATE"""
inner_join_df = db_df.join(df, lsuffix="_db", how='inner')
for index, row in inner_join_df.iterrows():
    formatted_prices = [{
        "date": datetime.today().strftime('%Y-%m-%d'), "price": float(row['priceNumber'])
    }]
    for rawprice in json.loads(row['shrPriceVos_db']):
        if not rawprice in formatted_prices:
            formatted_prices.append(rawprice)
    # print(json.dumps(formatted_prices))
    # print(row["shrPriceVos_db"])
    inner_join_df.at[index,'shrPriceVos'] = json.dumps(formatted_prices)
    
inner_join_df.to_csv(f"../../dataset/npa/krungthai_{datetime.today().strftime('%Y-%m-%d')}.csv")